# Install Spark


In [163]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.2.1/spark-2.2.1-bin-hadoop2.7.tgz
!tar xf spark-2.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu artful-security InRelease [83.2 kB]
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Get:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu artful-backports InRelease [74.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful-updates/universe amd64 Packages [150 kB]
Get:6 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 Packages [364 kB]
Fetched 761 kB in 1s (645 kB/s)
Reading package lists... Done


In [164]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"

In [165]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [166]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [167]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Load Data


In [10]:
datafile = files.upload()

Saving train_2v.csv to train_2v.csv


In [12]:
!ls

datalab			   spark-2.2.1-bin-hadoop2.7.tgz  test_2v.csv
spark-2.2.1-bin-hadoop2.7  spark-warehouse		  train_2v.csv


In [14]:
train_raw='./train_2v.csv'
train=sc.textFile(train_raw)

In [15]:
!ls

datalab			   spark-2.2.1-bin-hadoop2.7.tgz  test_2v.csv
spark-2.2.1-bin-hadoop2.7  spark-warehouse		  train_2v.csv


In [16]:
findspark.init()
findspark.find()

'/content/spark-2.2.1-bin-hadoop2.7'

In [17]:
!ls

datalab			   spark-2.2.1-bin-hadoop2.7.tgz  test_2v.csv
spark-2.2.1-bin-hadoop2.7  spark-warehouse		  train_2v.csv


#Previewing the data set

In [19]:
train.take(4)

['id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke',
 '30669,Male,3,0,0,No,children,Rural,95.12,18,,0',
 '30468,Male,58,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0',
 '16523,Female,8,0,0,No,Private,Urban,110.89,17.6,,0']

In [20]:
train = train.map(lambda line: line.split(","))

In [21]:
train.take(3)

[['id',
  'gender',
  'age',
  'hypertension',
  'heart_disease',
  'ever_married',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status',
  'stroke'],
 ['30669',
  'Male',
  '3',
  '0',
  '0',
  'No',
  'children',
  'Rural',
  '95.12',
  '18',
  '',
  '0'],
 ['30468',
  'Male',
  '58',
  '1',
  '0',
  'Yes',
  'Private',
  'Urban',
  '87.96',
  '39.2',
  'never smoked',
  '0']]

In [25]:
train.first()

['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [169]:
train.top(2)

[['id',
  'gender',
  'age',
  'hypertension',
  'heart_disease',
  'ever_married',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status',
  'stroke'],
 ['9998',
  'Male',
  '55',
  '0',
  '0',
  'Yes',
  'Govt_job',
  'Urban',
  '250.28',
  '38.6',
  'never smoked',
  '0']]

# Converting to Dataframe

In [26]:
from pyspark.sql import Row

In [27]:
train_df=train.map(lambda line: Row(id=line[0],
                                     gender =line[1],
                                    age=line[2],
                                    hypertension=line[3],
                                    heart_disease=line[4],
                                    ever_married=line[5],
                                    work_type=line[6],
                                    Residence_type=line[7],
                                    avg_glucose_level=line[8],
                                    bmi=line[9],
                                    smoking_status=line[10],
                                    stroke=line[11])).toDF()

In [30]:
train_df.show(3)

+--------------+---+-----------------+----+------------+------+-------------+------------+-----+--------------+------+---------+
|Residence_type|age|avg_glucose_level| bmi|ever_married|gender|heart_disease|hypertension|   id|smoking_status|stroke|work_type|
+--------------+---+-----------------+----+------------+------+-------------+------------+-----+--------------+------+---------+
|Residence_type|age|avg_glucose_level| bmi|ever_married|gender|heart_disease|hypertension|   id|smoking_status|stroke|work_type|
|         Rural|  3|            95.12|  18|          No|  Male|            0|           0|30669|              |     0| children|
|         Urban| 58|            87.96|39.2|         Yes|  Male|            0|           1|30468|  never smoked|     0|  Private|
+--------------+---+-----------------+----+------------+------+-------------+------------+-----+--------------+------+---------+
only showing top 3 rows



#Analyze the data type

In [33]:
train_df.printSchema()

root
 |-- Residence_type: string (nullable = true)
 |-- age: string (nullable = true)
 |-- avg_glucose_level: string (nullable = true)
 |-- bmi: string (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- heart_disease: string (nullable = true)
 |-- hypertension: string (nullable = true)
 |-- id: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: string (nullable = true)
 |-- work_type: string (nullable = true)



In [171]:
train_df.describe().show()

+-------+--------------+-----------------+------------------+------------------+------------+------+-------------------+-------------------+------------------+--------------+--------------------+---------+
|summary|Residence_type|              age| avg_glucose_level|               bmi|ever_married|gender|      heart_disease|       hypertension|                id|smoking_status|              stroke|work_type|
+-------+--------------+-----------------+------------------+------------------+------------+------+-------------------+-------------------+------------------+--------------+--------------------+---------+
|  count|         41938|            41938|             41938|             41938|       41938| 41938|              41938|              41938|             41938|         41938|               41938|    41938|
|   mean|          null|41.83929038099844|103.63264507583138|28.605038386852513|        null|  null|0.04311125947827746|0.08751013400734417|36816.046759502125|          null|0.

#Typecasting from String to Float

In [34]:
from pyspark.sql.types import *
train_df = train_df.withColumn("age", train_df["age"].cast(FloatType()))\
                   .withColumn("avg_glucose_level", train_df["avg_glucose_level"].cast(FloatType()))\
                   .withColumn("bmi", train_df["bmi"].cast(FloatType()))\
                   .withColumn("heart_disease", train_df["heart_disease"].cast(FloatType()))\
                   .withColumn("hypertension", train_df["hypertension"].cast(FloatType()))\
                    .withColumn("stroke", train_df["stroke"].cast(FloatType()))\
                    .withColumn("id", train_df["id"].cast(FloatType()))

In [61]:
train_df.printSchema()

root
 |-- Residence_type: string (nullable = true)
 |-- age: float (nullable = true)
 |-- avg_glucose_level: float (nullable = true)
 |-- bmi: float (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- heart_disease: float (nullable = true)
 |-- hypertension: float (nullable = true)
 |-- id: float (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: float (nullable = true)
 |-- work_type: string (nullable = true)



#Exploratory Data Analysis(EDA)

#Missing value Detection and Imputation

In [38]:
train_df.describe().show()

+-------+--------------+------------------+------------------+------------------+------------+------+-------------------+-------------------+------------------+--------------+-------------------+---------+
|summary|Residence_type|               age| avg_glucose_level|               bmi|ever_married|gender|      heart_disease|       hypertension|                id|smoking_status|             stroke|work_type|
+-------+--------------+------------------+------------------+------------------+------------+------+-------------------+-------------------+------------------+--------------+-------------------+---------+
|  count|         43401|             43400|             43400|             41938|       43401| 43401|              43400|              43400|             43400|         43401|              43400|    43401|
|   mean|          null| 42.21789400918109|104.48274998273718|28.605038386852513|        null|  null|0.04751152073732719|0.09357142857142857| 36326.14235023042|          null|0

As All other Columns has 43400 observation and bmi has 41938 observation,So Bmi column has 1462 no of missing values. As missing value is only 3% of entire data,dropped these values from the dataset to proceed further.

In [42]:
train_df=train_df.na.drop()

In [44]:
train_df.count(),test_df.count()

(41938, 18010)

# Deletion of unwanted column

As ID column contains unique value for each observation,it has no impact on dependant variable,Hence as a part of EDA dropped the column

In [63]:
train_new=train_df.drop("id")

In [64]:
test_new=test_df.drop("id")

In [65]:
train_new.describe().show()

+-------+--------------+-----------------+------------------+------------------+------------+------+-------------------+-------------------+--------------+--------------------+---------+
|summary|Residence_type|              age| avg_glucose_level|               bmi|ever_married|gender|      heart_disease|       hypertension|smoking_status|              stroke|work_type|
+-------+--------------+-----------------+------------------+------------------+------------+------+-------------------+-------------------+--------------+--------------------+---------+
|  count|         41938|            41938|             41938|             41938|       41938| 41938|              41938|              41938|         41938|               41938|    41938|
|   mean|          null|41.83929038099844|103.63264507583138|28.605038386852513|        null|  null|0.04311125947827746|0.08751013400734417|          null|0.015332156993657304|     null|
| stddev|          null|22.48365958008109|  42.2298142058113| 7.7

In [66]:
from pyspark.ml.feature import StringIndexer

In [67]:
plan_indexer = StringIndexer(inputCol = 'gender', outputCol = 'gender_new')
labeller_g= plan_indexer.fit(train_new)
Train1 = labeller_g.transform(train_new)

In [68]:
Train1.show(4)

+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+
|Residence_type| age|avg_glucose_level| bmi|ever_married|gender|heart_disease|hypertension| smoking_status|stroke|work_type|gender_new|
+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+
|         Rural| 3.0|            95.12|18.0|          No|  Male|          0.0|         0.0|               |   0.0| children|       1.0|
|         Urban|58.0|            87.96|39.2|         Yes|  Male|          0.0|         1.0|   never smoked|   0.0|  Private|       1.0|
|         Urban| 8.0|           110.89|17.6|          No|Female|          0.0|         0.0|               |   0.0|  Private|       0.0|
|         Rural|70.0|            69.04|35.9|         Yes|Female|          0.0|         0.0|formerly smoked|   0.0|  Private|       0.0|
+--------------+----+-----------------+----+----

# Dummy Variable Creation for categorical variables

In [69]:
plan_indexer = StringIndexer(inputCol = 'ever_married', outputCol = 'ever_married_new')
labeller_m= plan_indexer.fit(Train1)
Train1 = labeller_m.transform(Train1)

In [70]:
Train1.show(4)

+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+----------------+
|Residence_type| age|avg_glucose_level| bmi|ever_married|gender|heart_disease|hypertension| smoking_status|stroke|work_type|gender_new|ever_married_new|
+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+----------------+
|         Rural| 3.0|            95.12|18.0|          No|  Male|          0.0|         0.0|               |   0.0| children|       1.0|             1.0|
|         Urban|58.0|            87.96|39.2|         Yes|  Male|          0.0|         1.0|   never smoked|   0.0|  Private|       1.0|             0.0|
|         Urban| 8.0|           110.89|17.6|          No|Female|          0.0|         0.0|               |   0.0|  Private|       0.0|             1.0|
|         Rural|70.0|            69.04|35.9|         Yes|Female|          0.0|    

In [71]:
plan_indexer = StringIndexer(inputCol = 'Residence_type', outputCol = 'Residence_type_new')
labeller_r= plan_indexer.fit(Train1)
Train1 = labeller_r.transform(Train1)

In [72]:
plan_indexer = StringIndexer(inputCol = 'smoking_status', outputCol = 'smoking_status_new')
labeller_s= plan_indexer.fit(Train1)
Train1 = labeller_s.transform(Train1)

In [73]:
plan_indexer = StringIndexer(inputCol = 'work_type', outputCol = 'work_type_new')
labeller_w= plan_indexer.fit(Train1)
Train1 = labeller_w.transform(Train1)

In [74]:
Train1.show(4)

+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+----------------+------------------+------------------+-------------+
|Residence_type| age|avg_glucose_level| bmi|ever_married|gender|heart_disease|hypertension| smoking_status|stroke|work_type|gender_new|ever_married_new|Residence_type_new|smoking_status_new|work_type_new|
+--------------+----+-----------------+----+------------+------+-------------+------------+---------------+------+---------+----------+----------------+------------------+------------------+-------------+
|         Rural| 3.0|            95.12|18.0|          No|  Male|          0.0|         0.0|               |   0.0| children|       1.0|             1.0|               1.0|               1.0|          2.0|
|         Urban|58.0|            87.96|39.2|         Yes|  Male|          0.0|         1.0|   never smoked|   0.0|  Private|       1.0|             0.0|               0.0|         

Dropping the original column after creating dummy variables for categorical variables

In [75]:
Train1=Train1.drop('Residence_type','ever_married','gender','smoking_status','work_type')

In [79]:
Train1.show(3)

+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+
| age|avg_glucose_level| bmi|heart_disease|hypertension|stroke|gender_new|ever_married_new|Residence_type_new|smoking_status_new|work_type_new|
+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+
| 3.0|            95.12|18.0|          0.0|         0.0|   0.0|       1.0|             1.0|               1.0|               1.0|          2.0|
|58.0|            87.96|39.2|          0.0|         1.0|   0.0|       1.0|             0.0|               0.0|               0.0|          0.0|
| 8.0|           110.89|17.6|          0.0|         0.0|   0.0|       0.0|             1.0|               0.0|               1.0|          0.0|
+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------

In [102]:
Train1.describe().show()

+-------+-----------------+------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|              age| avg_glucose_level|               bmi|      heart_disease|       hypertension|              stroke|         gender_new|   ever_married_new| Residence_type_new|smoking_status_new|     work_type_new|
+-------+-----------------+------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|            41938|             41938|             41938|              41938|              41938|               41938|              41938|              41938|              41938|             41938|             41938|
|   mean|41.83929038099844|103.63264507583138|28.605038386852513|0.04311125947827746|0.0875101340073

# Selecting the features for Model Building

In [105]:
from pyspark.ml.feature import RFormula

In [106]:
formula=RFormula(formula="stroke~age+avg_glucose_level+bmi+heart_disease+hypertension+gender_new+ever_married_new+Residence_type_new+smoking_status_new+work_type_new",featuresCol="features",labelCol="label")

In [107]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)

In [108]:
train1.show()

+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+--------------------+-----+
| age|avg_glucose_level| bmi|heart_disease|hypertension|stroke|gender_new|ever_married_new|Residence_type_new|smoking_status_new|work_type_new|            features|label|
+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+--------------------+-----+
| 3.0|            95.12|18.0|          0.0|         0.0|   0.0|       1.0|             1.0|               1.0|               1.0|          2.0|[3.0,95.120002746...|  0.0|
|58.0|            87.96|39.2|          0.0|         1.0|   0.0|       1.0|             0.0|               0.0|               0.0|          0.0|(10,[0,1,2,4,5],[...|  0.0|
| 8.0|           110.89|17.6|          0.0|         0.0|   0.0|       0.0|             1.0|               0.0|               1.0|          0.0|(1

# Standardizing the Data

In [144]:
from pyspark.ml.feature import StandardScaler

In [145]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

In [146]:
scalerModel = scaler.fit(train1)

In [147]:
scaledData = scalerModel.transform(train1)

In [148]:
scaledData.show()

+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+--------------------+-----+--------------------+
| age|avg_glucose_level| bmi|heart_disease|hypertension|stroke|gender_new|ever_married_new|Residence_type_new|smoking_status_new|work_type_new|            features|label|      scaledFeatures|
+----+-----------------+----+-------------+------------+------+----------+----------------+------------------+------------------+-------------+--------------------+-----+--------------------+
| 3.0|            95.12|18.0|          0.0|         0.0|   0.0|       1.0|             1.0|               1.0|               1.0|          2.0|[3.0,95.120002746...|  0.0|[0.13343023582592...|
|58.0|            87.96|39.2|          0.0|         1.0|   0.0|       1.0|             0.0|               0.0|               0.0|          0.0|(10,[0,1,2,4,5],[...|  0.0|(10,[0,1,2,4,5],[...|
| 8.0|           110.89|17.6|          0

In [150]:
scaledData.select('scaledFeatures').show()
scaledData.select('label').show()

+--------------------+
|      scaledFeatures|
+--------------------+
|[0.13343023582592...|
|(10,[0,1,2,4,5],[...|
|(10,[0,1,2,6,8],[...|
|(10,[0,1,2,7,8],[...|
|[0.62267443385430...|
|(10,[0,1,2,8],[2....|
|(10,[0,1,2,8],[2....|
|[3.33575589564808...|
|(10,[0,1,2,7,8],[...|
|(10,[0,1,2,4,9],[...|
|(10,[0,1,2,8,9],[...|
|[3.51366287674931...|
|(10,[0,1,2,7],[1....|
|(10,[0,1,2,7,8],[...|
|(10,[0,1,2,7],[1....|
|[1.55668608463577...|
|(10,[0,1,2,6,8],[...|
|(10,[0,1,2,7],[1....|
|(10,[0,1,2,8,9],[...|
|[3.51366287674931...|
+--------------------+
only showing top 20 rows

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
+-----+
only showing top 20 rows




###Spliting The Data into training set and test set

In [153]:
(train_st, test_st) = scaledData.randomSplit([0.7, 0.3])

# Applying various Model On Standardized data


####Decission Tree Classifier

In [154]:
from pyspark.ml.classification import DecisionTreeClassifier

In [155]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="scaledFeatures", maxDepth=3)

In [157]:
dtModel = dt.fit(train_st)

In [158]:
predictions = dtModel.transform(test_st)

In [159]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.7176300432439026

Obtained Accuracy is 71.7% 

###Random Tree Classifier

In [161]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", featuresCol="scaledFeatures")

rfModel = rf.fit(train_st)

predictions = rfModel.transform(test_st)

evaluator.evaluate(predictions)

0.848359330050939

Obtained Accuracy is 84.8%

####Logistic Regression Model

In [162]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label", featuresCol="scaledFeatures", maxIter=10)
lrModel = lr.fit(train_st)
predictions = lrModel.transform(test_st)
evaluator.evaluate(predictions)

0.8265003018147066

In [0]:
Obtained Accuracy is 84.8%

In [0]:
Obtained Accuracy is 82.6%

# Applying Model on Non-Standardized Data

In [109]:
train1.select('features').show()
train1.select('label').show()

+--------------------+
|            features|
+--------------------+
|[3.0,95.120002746...|
|(10,[0,1,2,4,5],[...|
|(10,[0,1,2,6,8],[...|
|(10,[0,1,2,7,8],[...|
|[14.0,161.2799987...|
|(10,[0,1,2,8],[47...|
|(10,[0,1,2,8],[52...|
|[75.0,243.5299987...|
|(10,[0,1,2,7,8],[...|
|(10,[0,1,2,4,9],[...|
|(10,[0,1,2,8,9],[...|
|[79.0,57.08000183...|
|(10,[0,1,2,7],[37...|
|(10,[0,1,2,7,8],[...|
|(10,[0,1,2,7],[40...|
|[35.0,85.37000274...|
|(10,[0,1,2,6,8],[...|
|(10,[0,1,2,7],[42...|
|(10,[0,1,2,8,9],[...|
|[79.0,67.83999633...|
+--------------------+
only showing top 20 rows

+-----+
|label|
+-----+
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
|  0.0|
+-----+
only showing top 20 rows



###Spliting The Data into training set and test set

In [112]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

#Applying Various Classificatioon Model

####Applying  DEcission tree Classifier

In [103]:

from pyspark.ml.classification import DecisionTreeClassifier


In [111]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

In [113]:
dtModel = dt.fit(train_cv)

In [114]:
predictions = dtModel.transform(test_cv)

In [115]:
predictions.printSchema()

root
 |-- age: float (nullable = true)
 |-- avg_glucose_level: float (nullable = true)
 |-- bmi: float (nullable = true)
 |-- heart_disease: float (nullable = true)
 |-- hypertension: float (nullable = true)
 |-- stroke: float (nullable = true)
 |-- gender_new: double (nullable = true)
 |-- ever_married_new: double (nullable = true)
 |-- Residence_type_new: double (nullable = true)
 |-- smoking_status_new: double (nullable = true)
 |-- work_type_new: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [116]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8000428886226838

Obtained 80% accuracy

In [117]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1, 2, 6, 10])
             .addGrid(dt.maxBins, [20, 40, 80])
             .build())

In [118]:
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [119]:
cvModel = cv.fit(train_cv)

In [122]:
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)

numNodes =  7
depth =  2


In [120]:
predictions = cvModel.transform(test_cv)

In [121]:
evaluator.evaluate(predictions)


0.8202848396113405

Using Cross validation obtained 82% accuracy

####Applying RandomForest Classifier

In [124]:
from pyspark.ml.classification import RandomForestClassifier

In [125]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")


In [126]:
rfModel = rf.fit(train_cv)

In [127]:
predictions = rfModel.transform(test_cv)

In [128]:
evaluator.evaluate(predictions)

0.8594144224085631

Obtained 85.9% Accuracy

In [138]:
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [139]:
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [140]:
cvModel = cv.fit(train_cv)

In [141]:
predictions = cvModel.transform(test_cv)

In [142]:
evaluator.evaluate(predictions)

0.8563364242720017

Using cross validation obtained 85.6% accuracy

# Applying Logistic Model

In [130]:
from pyspark.ml.classification import LogisticRegression

In [131]:
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

In [135]:
lrModel = lr.fit(train_cv)

In [136]:
predictions = lrModel.transform(test_cv)

In [137]:
evaluator.evaluate(predictions)

0.8351708889775997

Obtained 83.5% accuracy

# Conclusion

Out of 3 classification model used here I obtained maximum accuracy(85.9%) using Random forest classification model